In [1]:
from typing import Annotated, Literal, Sequence, TypedDict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

c:\Users\Toshiba\anaconda3\envs\langgraph\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

c:\Users\Toshiba\anaconda3\envs\langgraph\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Toshiba\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(model_name = "Gemma2-9b-It")

In [6]:
llm.invoke('ISRO?')

AIMessage(content='ISRO stands for **Indian Space Research Organisation**. \n\nIt is the national space agency of India headquartered in Bengaluru.  \n\nHere are some key facts about ISRO:\n\n* **Established:** 15 August 1969\n* **Mission:** To conduct research and development in space technology and its applications.\n* **Achievements:**  ISRO has achieved many milestones, including:\n    * Launching its first satellite, Aryabhata, in 1975.\n    * Developing and launching its own launch vehicles, such as the Polar Satellite Launch Vehicle (PSLV) and Geostationary Satellite Launch Vehicle (GSLV).\n    * Placing India among the top spacefaring nations.\n    * Successful missions to the Moon (Chandrayaan-1, Chandrayaan-2) and Mars (Mangalyaan).\n* **Future Goals:** ISRO is working on ambitious projects, including:\n    *  Developing a human spaceflight program (Gaganyaan).\n    *  Exploring the Solar System with future missions.\n    *  Increasing the use of space technology for the bene

In [15]:
urls = [
    "https://python.langchain.com/v0.1/docs/modules/agents/",
    "https://blog.langchain.dev/what-is-an-agent/"
]

In [16]:
#load the data from web url using web base loader
docs = [WebBaseLoader(url).load() for url in urls]

In [17]:
docs

[[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/agents/', 'title': 'Agents | 🦜️🔗 LangChain', 'description': 'The core idea of agents is to use a language model to choose a sequence of actions to take.', 'language': 'en'}, page_content='\n\n\n\n\nAgents | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsAgentsQuickstartConceptsTypesAgentshow_toChainsMoreComponentsThis is documentation for LangChain v0.1, which is no longer actively maintained.For the current stabl

In [18]:
docs[0]

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/agents/', 'title': 'Agents | 🦜️🔗 LangChain', 'description': 'The core idea of agents is to use a language model to choose a sequence of actions to take.', 'language': 'en'}, page_content='\n\n\n\n\nAgents | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsAgentsQuickstartConceptsTypesAgentshow_toChainsMoreComponentsThis is documentation for LangChain v0.1, which is no longer actively maintained.For the current stable

In [19]:
docs[0][0].metadata

{'source': 'https://python.langchain.com/v0.1/docs/modules/agents/',
 'title': 'Agents | 🦜️🔗 LangChain',
 'description': 'The core idea of agents is to use a language model to choose a sequence of actions to take.',
 'language': 'en'}

In [20]:
docs_list = [item for sublist in docs for item in sublist]

In [23]:
docs_list

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/agents/', 'title': 'Agents | 🦜️🔗 LangChain', 'description': 'The core idea of agents is to use a language model to choose a sequence of actions to take.', 'language': 'en'}, page_content='\n\n\n\n\nAgents | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchModel I/OPromptsChat modelsLLMsOutput parsersRetrievalDocument loadersText splittersEmbedding modelsVector storesRetrieversIndexingCompositionToolsAgentsQuickstartConceptsTypesAgentshow_toChainsMoreComponentsThis is documentation for LangChain v0.1, which is no longer actively maintained.For the current stable

In [24]:
#Chunking

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100,chunk_overlap=5)

In [25]:
doc_splits = text_splitter.split_documents(docs_list)

In [26]:
doc_splits

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/agents/', 'title': 'Agents | 🦜️🔗 LangChain', 'description': 'The core idea of agents is to use a language model to choose a sequence of actions to take.', 'language': 'en'}, page_content='Agents | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/agents/', 'title': 'Agents | 🦜️🔗 LangChain', 'description': 'The core idea of agents is to use a language model to choose a sequence of actions to take.', 'language': 'en'}, page_content='Skip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain'),
 Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/modules/agents

In [28]:
#Retrieve operation using Chromadb
vectorstore = Chroma.from_documents(
    documents = doc_splits,
    collection_name = "RAG_CHROMA",
    embedding = embedding
)

In [29]:
retriever = vectorstore.as_retriever()

In [27]:
len(doc_splits)

27

In [8]:
def AI_ASSISTANT(state):
    pass

In [9]:
def RETRIEVER(state):
    pass

In [10]:
def REWRITER(state):
    pass

In [11]:
def GENERATE(state):
    pass

In [12]:
class AgentState:
    pass

In [13]:
#Create workflow
workflow = StateGraph(AgentState)
workflow.add_node("AI_ASTNT",AI_ASSISTANT)
workflow.add_node("RTVR",RETRIEVER)
workflow.add_node("RWRTR",REWRITER)
workflow.add_node("GNRTE",GENERATE)

In [14]:
#Edges
workflow.add_edge()
workflow.add_conditional_edges()

TypeError: StateGraph.add_edge() missing 2 required positional arguments: 'start_key' and 'end_key'

In [ ]:
#Compile workflow
app = workflow.compile()

In [ ]:
app.invoke("")